# Grade prediction with BERT embedding

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the reviews we scrapped in previous courses. We will try to predict wether it was positive or negative. 

Under the hood, the model is actually made up of two model.


1.   DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
2.   The next model, a basic Logistic Regression model from scikit learn will identify if the review was positive or negative



## Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 34.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

We'll import the dataset as we did in previous notebooks:

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')
dirpath = "drive" 
os.listdir(dirpath)

Mounted at /content/drive


['MyDrive',
 '.shortcut-targets-by-id',
 '.file-revisions-by-id',
 'Othercomputers',
 '.Trash-0']

In [ ]:
import os

dirpath = "drive/MyDrive"
data_file = "reviews.csv"
filepath = os.path.join(dirpath, data_file) 
reviews = pd.read_csv(filepath)
reviews.head()

,Unnamed: 0,page,titre,verbatim,date,note,reponse,date_experience,fournisseur,source
0,0,1,Aucun soucis particulier,Je paie ma facture tous les deux mois en fonct...,Il y a 17 heures,4,NaN,Date de l'expérience: 01 décembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot
1,1,1,Engie facture a ses clients des sommes…,Engie facture a ses clients des sommes exorbit...,Il y a un jour,1,"Bonjour Julien Blanco,\n\nPour des raisons de ...",Date de l'expérience: 26 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot
2,2,1,Facturation sur consommation d'un autre logement,Ils me facturent sur le pdl du logement au des...,ll y a 3 jours,1,"Bonjour BlooDz,\n\nPour des raisons de confide...",Date de l'expérience: 29 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot
3,3,1,un service client ou il est dur de…,un service client ou il est dur de comprendre ...,ll y a 3 jours,1,"Bonjour Ricanto77,\nPour des raisons de confid...",Date de l'expérience: 29 novembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot
4,4,1,Client d'ENGIE depuis longtemps toujours satis...,Excellente expérience avec ENGIE et une interl...,Il y a 24 minutes,5,NaN,Date de l'expérience: 01 décembre 2022,https://fr.trustpilot.com/review/engie.fr,trustpilot


In [ ]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37294 entries, 0 to 37293
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Unnamed: 0       37294 non-null  int64 
 1   page             37294 non-null  int64 
 2   titre            37294 non-null  object
 3   verbatim         36314 non-null  object
 4   date             37294 non-null  object
 5   note             37294 non-null  int64 
 6   reponse          7972 non-null   object
 7   date_experience  37294 non-null  object
 8   fournisseur      37294 non-null  object
 9   source           37294 non-null  object
dtypes: int64(3), object(7)
memory usage: 2.8+ MB


In the first place we should determine what are we considering a positive review. In this case we shall consider as positive every comment that had 4 or 5 as grade and negative otherwise.

In [ ]:
reviews['positive_or_negative'] = np.where(reviews["note"]>3, "positive", "negative")
reviews = reviews[["verbatim", "positive_or_negative"]].dropna()
reviews.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36314 entries, 0 to 37293
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   verbatim              36314 non-null  object
 1   positive_or_negative  36314 non-null  object
dtypes: object(2)
memory usage: 851.1+ KB


For computational purposes, we'll only use 1000 sentences which have less than 100 words in it.

In [ ]:
reviews = reviews[(reviews["verbatim"].str.split().str.len()< 100)]
reviews = reviews[:1000]

In [ ]:
sentences = reviews["verbatim"]
classification = reviews["positive_or_negative"]

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Right now, the variable model holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

# Model 1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
tokenized = sentences.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

### Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).


In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

Our dataset is now in the padded variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

(1000, 226)

### Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 226)

## Model 1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model! 

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the features variable, as they'll serve as the features to our logitics regression model.



In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

## Model 2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 1,000 sentences from the reviews training set).

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features, classification)

## [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

In [ ]:
# parameters = {'C': np.linspace(0.0001, 100, 20)}
# grid_search = GridSearchCV(LogisticRegression(), parameters)
# grid_search.fit(train_features, train_labels)

# print('best parameters: ', grid_search.best_params_)
# print('best scrores: ', grid_search.best_score_)

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. LogisticRegression(C=5.2)).

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

LogisticRegression()

## Evaluating Model 2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
lr_clf.score(test_features, test_labels)

0.928

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.547 (+/- 0.00)


So our model clearly does better than a dummy classifier